In [ ]:
# --- Full Runnable Script for Fine-Tuning Qwen ---

from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
import json
from datasets import Dataset, DatasetDict, concatenate_datasets

# Assume 'final_dataset' from Section 4 is available.

# --- 1. Load Qwen Model and Tokenizer ---
max_seq_length = 2048
dtype = None
load_in_4bit = True

print("--- Loading Qwen Model ---")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen2-1.5B-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

# --- 2. Configure QLoRA for Qwen ---
# It's crucial to correctly identify target modules for Qwen.
# We can use a helper function to find all linear layers and target them.
from unsloth.models.loader import _find_all_linear_names
target_modules, _ = _find_all_linear_names(model)
print(f"Identified target modules for Qwen: {target_modules}")


model = FastLanguageModel.get_peft_model(
    model,
    r=8,  # Using a lower rank for Qwen as per community findings
    lora_alpha=16,
    target_modules=target_modules,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)

# --- 3. The Training Loop with SFTTrainer ---
# We can reuse the TrainingArguments from the Llama 3.2 section,
# but we'll redefine it here for clarity and change the output directory.
training_args_qwen = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=10,
    max_steps=100, # Short run for demonstration
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=1,
    optim="paged_adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs_qwen",
    report_to="none",
)

trainer_qwen = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=final_dataset['train'],
    eval_dataset=final_dataset['test'],
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=training_args_qwen,
)

# Start the fine-tuning process for Qwen
print("--- Starting Qwen Fine-Tuning ---")
trainer_qwen_stats = trainer_qwen.train()
print("--- Fine-Tuning Complete ---")

# --- 4. Saving the Qwen Adapter ---
model.save_pretrained("qwen_risk_analyst_lora")
tokenizer.save_pretrained("qwen_risk_analyst_lora")
print("Qwen LoRA adapter saved successfully.")